In [1]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
import csv
DEBUG = 1
from datetime import datetime, timedelta
import dateparser
import calendar

In [2]:
def convertnumber(number):
    if 'k' in number:
        n=float(number.replace('k', '').replace(' ', ''))*1000
    elif 'm' in number:
        n=float(number.replace('m', '').replace(' ', ''))*1000000
    else:
        n=number
    return int(n)  

In [3]:
def scrolldown(browser):        
    src_updated = browser.page_source
    src = ""
    while  src != src_updated:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        src = src_updated
        src_updated = browser.page_source  

In [4]:
def connectchrome():

    # instantiate a chrome options object so you can set the size and headless preference
    options = Options()
    options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    driver.execute_script('window.scrollTo(0, 0);')
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    # and put it in the current directory
    chromedriver = "/home/youcef/Documents/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver     
    browser = webdriver.Chrome(options=options) 
    browser.get('https://www.quora.com/')
    time.sleep(2)
    return browser

In [5]:
def getTopic():
    file_question_topics = open((os.path.join('/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping'+'/topic3.txt')), mode='r', encoding='utf-8')
    lines=file_question_topics.readlines()
    line=lines[0]
    topic=line.strip()
    return str(topic.strip('\n'))
    

In [6]:
def crawlURL():
    # read topics form a file
    print(sys.path[0])
    file_question_topics = open((os.path.join('/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping'+'/topic3.txt')), mode='r', encoding='utf-8')
    cc=0
    driver=connectchrome()
    lines=file_question_topics.readlines()
    parsing_loop=True
    print(len(lines))
    while parsing_loop:
        line=lines[cc]
        cc+=1
        topic=line.strip()
        if (topic.startswith("#")):
            continue

       # give the url to scrape

        try:
            print('###################')
            print('starting new topic: ' + str(topic))
            url = "https://www.quora.com/topic/" + topic.strip('\n') + "/all_questions"
            print(str(url))
            driver.get(url)
        except Exception as e0:
            print('exception e0')
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e0).__name__, e0)
            continue     


        # define pause time for browser
        SCROLL_PAUSE_TIME = 2

        # get browser source
        html_source = driver.page_source
        question_count_soup = BeautifulSoup(html_source, 'html.parser')

        #  get total number of questions
        question_count_str = question_count_soup.find('a', attrs={'class': 'TopicQuestionsStatsRow'})

        if str(question_count_str)=='None':
            print('topic does not exist in Quora')
            continue

        question_count = question_count_str.contents[0].text 
        if 'k' in str(question_count):
            question_count = str(question_count).replace('k','')
            question_count = int(float(question_count)*1000)
            #print(type(question_count))
        print('number of questions for this topic : '+ str(question_count))

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        question_set = set()
        # infinite while loop, break it when you reach the end of the page or not able to scroll further.
        if int(question_count)>10:
            start_time_sd = time.time()
            max_time=  int(question_count)*0.25
            if int(question_count)> 8000:
                max_time=1800
            while True:
                i = 0
                # try to scroll 5 times in case of slow connection
                while True:
                    # Scroll down to one page length
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)

                    # get page height in pixels
                    new_height = driver.execute_script("return document.body.scrollHeight")

                    # break this loop when you are able to scroll futher
                    if new_height != last_height:
                        break
                    if i==1:
                        SCROLL_PAUSE_TIME = 4   
                    if i==2:
                        if int(question_count)>500 and int(question_count)<2000:
                            SCROLL_PAUSE_TIME = 7
                        elif int(question_count)>2000:
                            SCROLL_PAUSE_TIME = 15    
                        elif int(question_count)<500:
                            break
                    if i==3:
                        break
                    i += 1
            SCROLL_PAUSE_TIME = 2    
            # not able to scroll further, break the infinite loop
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            total_time=time.time() - start_time_sd
            if total_time>max_time:
                print('max time exceeded')
                break



        print('getting ressources ...')
        # get html page source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        # question_link is the class for questions
        question_link = soup.find_all('a', attrs={'class': 'question_link'}, href=True)

        # add questions to a set for uniqueness
        for ques in question_link:
            question_set.add(ques)


        # write content of set to a file called question_urls.txt
        questions_directory = 'topics/'
        os.makedirs('topics/', exist_ok=True)
        file_name = questions_directory + '/' + topic.strip('\n') + '_question_urls.txt'
        file_question_urls = open(file_name, mode='w', encoding='utf-8')
        writer = csv.writer(file_question_urls)
        for ques in question_set:
            link_url = "http://www.quora.com" + ques.attrs['href']
            #print(link_url)
            writer.writerows([[link_url]])

        #sleeping each 300 requests
        if cc % 300== 0:
            print('quitting chrome')
            driver.quit()
            time.sleep(6)
            driver=connectchrome()
        #loop_limit can be adjusted based on the number of requests we want to send 
        loop_limit=2
        if cc>loop_limit:
            parsing_loop=False

    # finish time
    end_time = datetime.datetime.now()
    print(end_time-start_time)

In [7]:
def extractQuestionTitle(driver):
    try:
        question_title = driver.find_element_by_xpath('.//span[contains(@class, "rendered_qtext")]').text
    except:
        question_title = "error"
    return question_title

In [8]:
def extractAnswerCount(driver):
    try:
        answer_count = driver.find_element_by_xpath("//div[@class = 'answer_count']")
        text_count = answer_count.text.split()
        count = int((text_count[0].split('+'))[0])
    except:
        count = "error"
    return count

In [9]:
def extractTopics(driver):
    try:
        topic_list = []
        topics = driver.find_elements_by_xpath(".//span[@class = 'TopicName TopicNameSpan']")
        for topic in topics:
            topic_list.append(topic.text)
    except:
        topic_list = []
    return topic_list

In [10]:
def scrollDownPage(driver):
    current_scroll_position, new_height= 0, 1
    speed = 10
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [11]:
def expandAllAnswers(driver):
    hidden_list = driver.find_elements_by_xpath(".//div[@class='ui_qtext_truncated ui_qtext_truncated_regular']/div[@class='ui_qtext_truncated_text']")
    #print('Number of hidden answer:',len(hidden_list))
    for hidden in hidden_list:
        hidden = hidden.find_element_by_xpath(".//a[@class='ui_qtext_more_link']")
        webdriver.ActionChains(driver).move_to_element(hidden).click(hidden).perform()

In [12]:
def extractRespond(driver):
    check_ans_count = 0
    answers_list = {}
    name_list = []
    upvote_list = []
    ans_list = []
    date_list = []
    view_list = []
    error_list = []
    question_title = ''
    check_ans_count = 0
    try: 
        respond_list = driver.find_elements_by_xpath(".//div[contains(@class, 'AnswerBase') and contains(@class, 'AnswerBase') and contains(@class, 'Toggle') and contains(@class, 'Answer')]")
        for respond in respond_list:
            # Name
            ans = {}
            try:
                user_name = respond.find_element_by_xpath('.//a[@class = "user"]')
                name_list.append(user_name.text)
                ans['user_name'] = user_name.text
            except Exception:
                user_name = respond.find_element_by_xpath('.//span[contains(@class, "anon_user")]')
                name_list.append(user_name.text)
                ans['user_name'] = user_name.text
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                name_list.append('')
                ans['user_name'] = 'error'
                print(error_list[-1])

            # Date
            try:
                date = respond.find_element_by_xpath(".//a[@class = 'answer_permalink']")
                date_split = date.text.split('Answered ')
                question_date = date_split[1]
                date_list.append(question_date)
                ans['date'] = question_date
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                date_list.append('')
                ans['date'] = 'error'
                print(error_list[-1])

            # Features that show after expand
            try:
                expanded_answer = respond.find_element_by_xpath(".//div[contains(@class,'ExpandedAnswer')]")
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                print(error_list[-1])
                continue

            # Answer
            try:
                answer = expanded_answer.find_element_by_xpath(".//div[@class='ui_qtext_expanded']/span[@class='ui_qtext_rendered_qtext']")
                ans_list.append(answer.text)
                ans['answer'] = answer.text
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                ans_list.append('')
                ans['answer'] = 'error'
                print(error_list[-1])

            # Answer upvote
            try:
                upvote = expanded_answer.find_element_by_xpath(".//a[@class='VoterListModalLink']")
                res = [int(i) for i in upvote.text.split() if i.isdigit()]
                upvote_list.append(res[0])
                ans['upvote'] = res[0]
            except Exception:
                upvote_list.append(0)
                ans['upvote'] = 0
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                upvote_list.append(0)
                ans['upvote'] = 'error'
                print(error_list[-1])

            # Answer view
            try:
                answer_footer = expanded_answer.find_element_by_xpath(".//div[contains(@class, 'AnswerFooter')]")
                view_text = (answer_footer.text.split())[0]
                if 'k' in view_text:
                    view = float((view_text.split('k'))[0]) * 1000
                elif 'm' in view_text:
                    view = float((view_text.split('m'))[0]) * 1000000
                else:
                    view = int(view_text)
                view_list.append(view)
                ans['view'] = view
            except Exception as e:
                error_list.append([e, question_title, check_ans_count])
                view_list.append(0)
                ans['view'] = 'error'
                print(error_list[-1])

            print()    
            check_ans_count += 1
            print(type(ans))
            if check_ans_count > len(answers_list):
                answers_list[str(len(answers_list))] = ans
        print("Respond collected:", check_ans_count)

        return answers_list
    
    except Exception as e:
        error_list.append([e, question_title, check_ans_count])
        print(error_list[-1])
        return check_ans_count, name_list, upvote_list, ans_list, date_list, view_list, error_list
    
    

In [13]:
def scrollDownPageFaster(driver):
    current_scroll_position, new_height= 0, 1
    speed = 5
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [14]:
def getQuestionAddedBy(driver):
    try:
        last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
        user_name = last_item.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except:
        try:
            last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
            user_name = last_item.find_element_by_xpath(".//a[@class = 'user']").text
        except:
            return
    return user_name

In [15]:
def extractCreateTime(driver):
    try:
        last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
        question_data = last_item.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        question_data_split = question_data.text.split('· ')
        question_date = question_data_split[1]
    except:
        question_date = ''
    return question_date

In [16]:
def extractNoOfFollower(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
    except:
        no_of_follower = 'error'
    return no_of_follower

In [17]:
def extractNoOfView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_view = 'error'
    return no_of_view

In [18]:
def extractQuestion(driver, url):
    # Extract Question
    data = {}
    scrollDownPage(driver)  
    expandAllAnswers(driver)
    data['question_title'] = extractQuestionTitle(driver)
    data['answer_count'] = extractAnswerCount(driver)
    data['topic_list'] = extractTopics(driver)
    data['respond'] = extractRespond(driver)
    driver.get(url+'/log')
    scrollDownPageFaster(driver)
    data['question_author'] = getQuestionAddedBy(driver)
    data['create_time'] = extractCreateTime(driver)
    data['no_of_follower'] = extractNoOfFollower(driver)
    data['no_of_view'] = extractNoOfView(driver)
    return data

In [19]:
def crawlQuestionData(file):
    start_time = time.time()
    #if (DEBUG): #print ("In crawlQuestionData...")
    
    # Open question url file
    file_question_urls = open(file, mode = 'r')
    topic = str(getTopic())
    topic = topic + "_answers.txt"

    #file_users = open(os.path.join(sys.path[0]+"users.txt", mode = 'a'))
    current_lines = file_question_urls.readlines()
    i=-1
    browser= connectchrome()
    k=0
    loop=True
    #for current_line in current_lines:
    dictdata = {}
    while  loop :
        try:
            current_line= current_lines[k]
        except:
            break
        print('processing question number  : '+ str(k))
        k+=1
        if '/unanswered/' in str(current_line):
            print('answer is unanswered')
            continue     
        print ("*************************************************************************")
        if (DEBUG): print(current_line)
        question_id = current_line
        browser.get(current_line)
        data = extractQuestion(browser, current_line)
        print(type(data))
        print(data)
        dictdata[str(len(dictdata))] = data
        print(type(dictdata))
        
        j = json.dumps(dictdata)
        # we save all retrieved answers into answers.txt file
        with open(topic, 'w') as file_answers:
            file_answers.write(j)
            file_answers.close()
    
        print ("*************************************************************************")
        # In our usecase 74256 is the loop limit value
        if k>=20000 :
           loop= False
    print("--- %s seconds ---" % (time.time() - start_time))        
    browser.quit()

In [20]:
def main():
    print(os.getcwd())
    #crawlURL()
    # We merged all Questions urls crawled by 1-Questions_URLs_crawler.py into one file (quora_urls.txt)
    crawlQuestionData((os.path.join(os.getcwd()+"/topics/" + getTopic() + "_question_urls.txt")))
    return 0

if __name__ == "__main__": main()    

/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping
processing question number  : 0
*************************************************************************
http://www.quora.com/Why-was-Jesus-called-King-of-the-Jews-and-yet-people-who-believe-in-Judaism-do-not-believe-in-him

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]

<class 'dict'>
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 1]

<class 'dict'>
Respond c


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'If you are saved and you kill yourself, do you go to heaven?', 'respond': {'0': {'view': 25, 'answer': 'According to my understanding and beliefs - once you are saved you are always saved, no matter what.\nSuicide is a sin, and the Catholics and some other Christian denominations believe you absolutely CAN lose your salvation if you commit suicide. On the Catholic side they beleive they are the representation of God as gatekeepers but of course, Jesus told us that not through man but through Christ may you know God. Those other churches I mentioned, mainly in the USA, are run by judgmental and controlling men desperate to hold on to power over people\'s lives - same with those pagans I mentioned before - the Catholics ←opinions..mostly. Suicide is definitely a sin though.\nIn God\'s eyes you are no more special than another person "God is no respecter of persons" and all sin is equally disgusting to God - it is hum

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]
Respond collected: 0
<class 'dict'>
{'question_title': 'What is a Teil tree in the Bible?', 'respond': {}, 'question_author': 'Vlad Racovita', 'answer_count': 2, 'create_time': 'Apr 14, 2020', 'topic_list': [], 'no_of_follower': 2, 'no_of_view': '1,561'}
<class 'dict'>
*************************************************************************
processing question number  : 10
*************************************************************************
http://www.quora.com/Jesus-the-Christ-is-said-to-have-cured-Peters-mother-in-law-of-a-fever-If-she-had-Coronavirus-2020-c

[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 0]

<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'What is "Basanti Puja"?', 'respond': {'0': {'view': 8, 'answer': 'If I am not wrong, ‘Basanti Puja’ is worshipping Goddess around Ram Navami in spring season. In Bengal it is called Basanti Durga Puja.\nIn north India on the day of Vasant Panchami (before Holi) tha is at the beginning of Spring season, Goddess Saraswati is worshipped.', 'upvote': 1, 'user_name': 'Prafulla Pant (प्रफुल्ल पंत)', 'date': 'Apr 13, 2020'}}, 'question_author': 'Sandipan Hore', 'answer_count': 2, 'create_time': 'Apr 13, 2020', 'topic_list': [], 'no_of_follower': 0, 'no_of_view': '393'}
<class 'dict'>
*************************************************************************
processing question number  : 14
answer is unanswered
pr

[IndexError('list index out of range',), '', 5]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 5]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 5]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 5]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 5]
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector"

...


<class 'dict'>

<class 'dict'>

<class 'dict'>
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 3]

<class 'dict'>

<class 'dict'>
[NoSuchElementException('no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class,\'ExpandedAnswer\')]"}\n  (Session info: headless chrome=81.0.4044.92)', None, None), '', 5]
Respond collected: 5
<class 'dict'>
{'question_title': 'Believers, Do you think that COVID-19 is a sign of end times?', 'respond': {'0': {'view': 67, 'answer': "Read these —-\nSpirituality and the bat plague?\nWhat are Christian healers saying about the lockdown-\nDavid Hague's answer to What are all the Christian faith healers like Benny Hinn and company doing about the current pandemic?\n.\nIs the World ready for the truth behind the Bat plague ?\nDavid Hague's answer to Is the world r

Respond collected: 0
<class 'dict'>
{'question_title': 'Are government people going to heaven for disobedience?', 'respond': {}, 'question_author': 'Tshepang Sebothe', 'answer_count': 'error', 'create_time': 'Apr 13, 2020', 'topic_list': [], 'no_of_follower': 1, 'no_of_view': '810'}
<class 'dict'>
*************************************************************************
processing question number  : 1647
*************************************************************************
http://www.quora.com/Is-Easter-just-a-widespread-fan-fiction-celebration


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'Is Easter just a widespread fan fiction celebration?', 'respond': {'0': {'view': 32, 'answer': 'Easter is the celebration that takes place where there is a spring and plant life is renewed and a growing season begins after winter. Vernal equinox (about March 21): when day and night of equal length, marking the start of spring.\nJesus said, “Hey mom, hang on to that choc


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'After coronavirus, do you think that the world will become less religious and more scientific?', 'respond': {'0': {'view': 10, 'answer': 'Yes . People faith on religion will lesser on the other hand they will start believing in medical staff even they are started .\nPeople are nowadays more running behind scientific reason than religious beliefs but may be some Muslim community will not change their mindset because they think that all this suffering because of alllah\nSo except muslim community all other religions will face questions .\nHowever; every religion will change their views about corona they will try to escape.', 'upvote': 0, 'user_name': 'Anjani Kumar Sharma', 'date': 'Apr 14, 2020'}}, 'question_author': 'Sandeep Rajak', 'answer_count': 1, 'create_time': 'Apr 14, 2020', 'topic_list': [], 'no_of_follower': 1, 'no_of_view': '271'}
<class 'dict'>
*************************************************************


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'Are there any modern-day living descendants of historic religious figures like Mohammed?', 'respond': {'0': {'view': 23, 'answer': 'As a matter of fact many thousands of Muslims claim to be descended from the Prophet Mohammed though his daughter Fatima who married his cousin Ali.\nThe Kings of Morocco and Jordan and the Agha Khan are among them.\nThe Zhengy Dao is a Daoist sect founded by Zhang Daoling in AD 142 lead by the Celestial Masters who claim to be descended from Zhang Daoling.\nList of Celestial Masters - Wikipedia .\nIf Confucianism counts as a religion instead of a secular philosophy, then the many thousands of people who claim to be descendants of Confucius claim to be descendants of a historic religious figure.\n“Whirling Dervishes” are Sufi mystics. Their sect was founded by Jelaladdin Rumi (1207–1273), and the Mewlewi Sufi order has been led by the Family of Rumi since 1284. It has also been claimed


<class 'dict'>
Respond collected: 1
<class 'dict'>
{'question_title': 'Why is it that among Muslims, the father is the head of the family, while among Jews, the mother decides ancestry?', 'respond': {'0': {'view': 55, 'answer': 'Traditionally in Judaism, the father is also the head of the family. In Orthodox Judaism, Jewish status passes through the mother. However, Jewish lineage, passes through the father. As to why Jewish status historically was traced maternally, it is difficult to say. It may be rooted in traditional interpretations of certain biblical passages. Or it may be that prior to DNA testing, you always knew who a child’s mother was, but not necessarily the father.', 'upvote': 1, 'user_name': 'Benjamin Ferrandi', 'date': 'Apr 14, 2020'}}, 'question_author': 'Thomas Jose Joseph', 'answer_count': 1, 'create_time': 'Apr 13, 2020', 'topic_list': [], 'no_of_follower': 1, 'no_of_view': '250'}
<class 'dict'>
**********************************************************************